# 데이터 수집 – 한 페이지 크롤링

에누리 사이트에서 제품별로 용량, 크기 같은 조건을 설정해서 검색하여

세탁기 제품의 가격과 정보를 수집한다.  

크롤링에 필요한 라이브러리들을 임포트


In [1]:
from bs4 import BeautifulSoup 
import time
import pandas as pd

In [2]:
# selenium4
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

options = Options()
# 브라우저 꺼짐 방지 코드
options.add_experimental_option("detach", True)
# 불필요한 에러 메세지 제거 코드
# options.add_experimental_option("excludeSwitched",["enable-logging"]);
# service = Service(ChromeDriverManager().install()) # [ 패키지부터 에러발생 ]


#driver = webdriver.Chrome(options=options)
#driver.implicitly_wait(3)


### 에누리 검색 페이지 접속

In [3]:
# 예제 6-1 selenium으로 에누리 검색 결과 URL에 접속
from selenium import webdriver


# 에누리 사이트에서 실제 '세탁기' 검색 후 url 확인
#url = "https://www.enuri.com/list.jsp?cate=0502&from=search&keyword=%EC%84%B8%ED%83%81%EA%B8%B0" 
#driver.get(url)

### 에누리 검색 웹 페이지에서 상품 정보 가져오기  
웹 페이지의 HTML 정보 가져오기

In [7]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

### 6.2.3 여러 페이지에 걸친 상품 정보 수집

In [9]:
# 예제 6-10 상품 정보 태그에서 원하는 정보를 추출하는 함수
def get_prod_items(prod_items):
    prod_data = []
    for prod_item in prod_items:
         # ① 상품명 가져오기
        try:
            title = prod_item.select('.item__model > a')[0].text.strip()
        except:
            title = ''
        # ② 스펙 목록 가져오기
        try: 
             spec_list = prod_item.select(".item__attr")[0].text.strip()+prod_item.select("div.item__summ")[0].text.strip()
            
        except:
            spec_list = ''
        # ③ 가격 정보 가져오기 
        try: 
            price = prod_item.select('div.opt--price > span')[0].text.strip().replace(",","")
        except:
            price = 0
        prod_data.append([title, spec_list, price])
    return prod_data

In [10]:
# 예제 6-12 에누리 검색 URL을 만들어주는 함수 
def get_search_page_url(keyword, page):
    return 'https://www.enuri.com/list.jsp?cate=0502&from=search&keyword={}&page={}'.format(keyword, page)

keyword = '세탁기'
page = 1
url = get_search_page_url(keyword, page)
print(url)

https://www.enuri.com/list.jsp?cate=0502&from=search&keyword=세탁기&page=1


### 셀리니움을 이용한 에누리 사이트 크롤링

In [11]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook

# [selenium4]
driver = webdriver.Chrome(options=options)

# 암묵적으로 웹 자원 로드를 위해 3초까지 기다림
driver.implicitly_wait(3)
keyword = '세탁기'
total_page = 3
prod_data_total = []
# 진행 정도를 표현하는 tqdm을 적용 ==> 오류발생
# for page in tqdm_notebook(range(1, total_page + 1)):
for page in range(1, total_page + 1):
    
    # ① 검색 페이지 이동
    url = get_search_page_url(keyword, page)
    driver.get(url)
    # 페이지가 로딩 완료되기 위한 시간으로 5초를 할당
    time.sleep(5)
    
    # ② 현재 페이지의 HTML 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # ③ 상품 정보 추출
    prod_items = soup.select('div.goods-list > div.goods-cont.type--list > ul > li')
    prod_item_list = get_prod_items(prod_items)
    # ④ 추출 데이터 저장
    prod_data_total = prod_data_total + prod_item_list


### 크롤링을 통해 수집한 데이터 저장 

In [12]:
# 예제 6-16 데이터 저장
import pandas as pd
data = pd.DataFrame(prod_data_total)
data.columns = ['상품명', '스펙 목록', '가격'] #각각의 컬럼들을 지정을 해준다.
data.to_excel('./files/crawling_result.xlsx', index = False)